In [143]:
import pandas
import requests
import datetime
import time
import json

In [138]:
etherscan_api_key = "SU775DFF8GEGMER3B8ZPK3QZIJWGIBE45T"
url_endpoint = "https://api.etherscan.io/api"

In [139]:
#Starting Point for analysis
starting_block = 4826779 #Dec 31st 2017 start
ending_block = 4832685 #Dec 31st 2017 end
hex_starting_block = hex(starting_block)
hex_end_block = hex(ending_block)
get_blocks_by_range = 100

In [140]:
def tx_cost_in_ETH(gasused, gasprice):
    '''
    Does the conversion to get the actual transaction fee (in ETH) for each ethereum transaction.
    '''
    wei_in_ETH = 1000000000000000000
    cost_in_eth = (gasused * gasprice)/float(wei_in_ETH)
    return cost_in_eth

In [141]:
#Datapoints per transaction needed
# Timestamp
# Unique Hash
# Account Sending
# Account Receiving (May be smart contract)
# Tx Fee
# Size of Data (I don't see this here... I'll skip it for now)
# Network Name (duh)
# Transaction Status
# Value of Tx

In [146]:
#Transactions processed for this group of blocks
processed_txs = []

#Loop for all transactions
for i in range(10):
    #Get Transactions in a Block
    #https://github.com/ethereum/wiki/wiki/JSON-RPC#eth_gettransactionbyhash
    next_block = starting_block + i
    hex_next_block = hex(next_block)
    
    print "Next Block is " + str(next_block)
    print datetime.datetime.now()
    
    #Go and grab all transactions in the block
    params = {"module":"proxy", "action":"eth_getBlockByNumber", "tag" : hex_starting_block, "boolean":"true", "apikey": etherscan_api_key}
    resp = requests.get(url_endpoint, params=params)
    resp_dict = json.loads(resp.content) #Get a string, must eval to a dict

    #Timestamp

    timestamp = int(resp_dict['result']['timestamp'],0)
    unix_timestamp = int(resp_dict['result']['timestamp'],0)
    timestamp = datetime.datetime.fromtimestamp(timestamp).strftime("%Y-%m-%d %H:%M:%S")

    #Every other feature will be found in the transactions

    transactions = resp_dict['result']['transactions']
    transactions

    for i in transactions:
        #Initialize the next transaction
        next_tx = {'datetime': timestamp, 'unixtimestamp': unix_timestamp, 'network': "Ethereum"}

        #Unique Hash
        next_tx['txhash'] = i['hash']

        #Account Sending
        next_tx['sender'] = i['from']

        #Account Receiving
        next_tx['receiver'] = i['to']

        #Tx Fee
        #Can't evaluate the fee until we get the receipt info, which is a separate call.
        #Oh Bother.
        next_tx['gasprice'] = int(i['gasPrice'],0)    

        processed_txs.append(next_tx)

#Now I'm going through and grabbing the transaction hashes
for i in processed_txs:
    tx_hash = i['txhash']

    params = {"module":"proxy", "action":"eth_getTransactionReceipt", "txhash" : tx_hash, "apikey": etherscan_api_key}
    resp = requests.get(url_endpoint, params=params)
    resp_dict = json.loads(resp.content)

    i["gasused"] = int(resp_dict['result']['gasUsed'],0)
    i['status'] = int(resp_dict['result']['status'],0)
    i['txfee'] = tx_cost_in_ETH(i['gasused'], i['gasprice'])

Next Block is 4826779
2018-01-01 14:58:00.392000
Next Block is 4826780
2018-01-01 14:58:27.358000
Next Block is 4826781
2018-01-01 14:59:21.439000
Next Block is 4826782
2018-01-01 15:00:39.914000
Next Block is 4826783
2018-01-01 15:02:27.076000
Next Block is 4826784
2018-01-01 15:04:44.514000
Next Block is 4826785
2018-01-01 15:07:33.848000
Next Block is 4826786
2018-01-01 15:10:43.125000
Next Block is 4826787
2018-01-01 15:14:21.448000
Next Block is 4826788
2018-01-01 15:18:30.856000


In [147]:
df = pandas.DataFrame.from_dict(processed_txs)

df.to_csv("20171231_transactions.csv")

2018-01-01 13:55:52.019000
